# Use pymatgen


In [5]:
#read an write cif files
from pymatgen.io.cif import CifParser, CifWriter
cif_file = "../cif_files/C12O12Rh4.cif"
cif_out = "C12O12Rh4_test.cif"
parser = CifParser(cif_file)
structure = parser.get_structures()
cif_writer = CifWriter(parser.get_structures()[0])
cif_writer.write_file(cif_out)
parser2 = CifParser(cif_out)
structure2 = parser2.get_structures()[0]

In [12]:
from pymatgen.io.feff.inputs import Atoms, Potential, Header
out_inp = "C12O12Rh4.inp"
out_header = Header.from_cif_file(cif_file, source=cif_file, comment=parser.get_bibtex_string())
out_atoms = Atoms(structure2,absorbing_atom="Rh",radius=6)
out_potential = Potential(structure2,absorbing_atom="Rh")
out_atoms.write_file(filename=out_inp)
#out_header.write_file(filename=out_inp)
#out_potential.write_file(filename=out_inp)

In [16]:
from larch.xafs.feffrunner import feff6l
feff_dir = "./temp_feff"
feff6l(folder = feff_dir, feffinp=out_inp)

 : ======== running Feff module C:\Users\scman1\AppData\Local\Programs\Python\Python39\lib\site-packages\larch\bin\win64\feff6l.exe ========
 : Feff 6L.02
 : 
 : Calculating potentials and phases...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : free atom potential and density for atom type    2
 : free atom potential and density for atom type    3
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : overlapped potential and density for unique potential    2
 : overlapped potential and density for unique potential    3
 : muffin tin radii and interstitial parameters
 : phase shifts for unique potential    0
 : phase shifts for unique potential    1
 : phase shifts for unique potential    2
 : phase shifts for unique potential    3
 : Preparing plane wave scattering amplitudes...
 : nncrit in prcrit       9
 : Searching for paths...
 : Rmax  5.0000  k

Attribute,Type
Feff8l_modules,tuple
run,method
folder,str
feffinp,str
verbose,bool
message_writer,NoneType
mpse,bool
resolved,NoneType
threshold,list
chargetransfer,list


In [14]:
dir(parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cif',
 '_frac_tolerance',
 '_get_structure',
 '_occupancy_tolerance',
 '_parse_symbol',
 '_sanitize_data',
 '_site_tolerance',
 '_unique_coords',
 'as_dict',
 'feature_flags',
 'from_string',
 'get_bibtex_string',
 'get_lattice',
 'get_lattice_no_exception',
 'get_magsymops',
 'get_structures',
 'get_symops',
 'has_errors',
 'parse_magmoms',
 'parse_oxi_states',
 'symmetry_operations',
 'warnings']

In [ ]:
parser.as_dict()

In [ ]:
print(parser.get_bibtex_string())

In [ ]:
dir(parser)

In [ ]:
parser.get_structures()